In [ ]:
### Try using a neural network to learn a function just from random space points and the exact value
### See how the error goes down as the runtime, size of network, methodology change

# Laplace's equation
# u(x,y) = 4.0*(x - 1.0)**2 - 4.0*(y + 0.5)**2
#  in a polar region r = 0.75*cos(theta) + 1.1

# High frequency 
# u(x,y) = sin( 4.2*x ) cos( 7.4*y).^2

# 

In [ ]:
import torch
import numpy as np
import time
import math

import DRLPDE.create as create
import DRLPDE.neuralnets as neuralnets
import DRLPDE.train as train
import DRLPDE.parameters_solver as solver

import examples.hybrid as param

torch.set_default_dtype(torch.float32)

use_cuda=torch.cuda.is_available()
dev = torch.device("cuda:0" if use_cuda else "cpu")

In [ ]:

# Organize parameters
input_dim = [param.x_dim, param.t_dim, param.hyper_dim]

input_range = param.boundingbox + param.t_range + param.hyper_range

there_are_walls = any(param.list_of_walls)
there_are_solids = any(param.solid_walls)
there_are_inletoutlets = any(param.inlet_outlet)
there_are_meshes = any(param.mesh)
unsteady = bool(param.t_dim)

#num = solver.numpts
num = solver.numpts
numbatch = solver.numbatch
resample_every = solver.resample_every
reweight_every = solver.reweight_every
trainingsteps = solver.trainingsteps
print_every = solver.print_every

MyNeuralNetwork = neuralnets.FeedForwardNN

var_nn = {'depth': solver.nn_depth,
        'width': solver.nn_width,
        'x_dim': input_dim[0],
        'is_unsteady': False,
        'output_dim': param.output_dim
        }

if param.loadmodel:
    model = MyNeuralNetwork(**var_nn).to(dev)
    model.load_state_dict(torch.load("savedmodels/" + param.loadmodel + ".pt"))
    print("Using model from savedmodels/" + param.loadmodel + ".pt")
else:
    model = MyNeuralNetwork(**var_nn).to(dev)

optimizer = torch.optim.Adam(model.parameters(), 
                                lr=solver.learningrate, 
                                betas=solver.adambeta, 
                                weight_decay=solver.weightdecay)


# Define the domain through its boundaries
Domain = create.SpaceDomain(param.boundingbox, 
                            param.list_of_walls, 
                            param.solid_walls, 
                            param.inlet_outlet, 
                            param.list_of_periodic_ends, 
                            param.mesh, 
                            param.init_con)

### Create Interior points and organize into batches
IntPoints = create.InteriorPoints(num, Domain, input_dim, input_range)
IntPoints_batch = torch.utils.data.DataLoader(IntPoints, batch_size=numbatch, shuffle=True)

### Create Dirichlet Wall points and organize into batches
if there_are_walls:
    num_wall = create.numBCpoints(num, input_dim, Domain, Domain.wall)
    BCPoints = create.BCPoints(num_wall, Domain, input_dim, input_range)

    BCPoints_batch = torch.utils.data.DataLoader(BCPoints, batch_size=numbatch, shuffle=True)

### Create Solid Wall points and organize into batches
if there_are_solids:
    num_solid = create.numBCpoints(num, input_dim, Domain, Domain.solid)
    SolidPoints = create.SolidWallPoints(num_solid, Domain, input_dim, input_range)
    SolidPoints_batch = torch.utils.data.DataLoader(SolidPoints, batch_size=numbatch, shuffle=True)

### Create Inlet/Outlet points and organize into batches
if there_are_inletoutlets:
    num_inout = create.numBCpoints(num, input_dim, Domain, Domain.inletoutlet)
    InletOutletPoints = create.InletOutletPoints(num_inout, Domain, input_dim, input_range)
    InletOutletPoints_batch = torch.utils.data.DataLoader(InletOutletPoints, batch_size=numbatch, shuffle=True)

if there_are_meshes:
    MeshPoints = create.MeshPoints(num_mesh, Domain.mesh[0], model)
    MeshPoints_batch = torch.utils.data.DataLoader(MeshPoints, batch_size=numbatch, shuffle = True)

### Create Initial Condition points and organize into batches
if unsteady:
    num_ic = create.numICpoints(num, input_dim)
    ICPoints = create.ICPoints(num_ic, input_dim, input_range, Domain, param.init_con)
    ICPoints_batch = torch.utils.data.DataLoader(ICPoints, batch_size=numbatch, shuffle=True)

### Random points for calculating L2 error through Monte Carlo Integration
ErrorPoints = create.ErrorPoints(num_error, Domain, input_dim, input_range)
ErrorPoints_batch = torch.utils.data.DataLoader(ErrorPoints, batch_size=numbatch, shuffle = True)

### Method type
if solver.method == 'autodiff':
    import DRLPDE.autodiff as method
    var_train = {'diffusion': param.diffusion,
                    'forcing': param.forcing,
                    'x_dim': input_dim[0]
                }

elif solver.method == 'stochastic':
    import DRLPDE.stochastic as method
    var_train = {'diffusion': param.diffusion,
                'forcing': param.forcing, 
                'x_dim': input_dim[0],
                'domain': Domain,
                'dt': solver.dt,
                'num_ghost': solver.num_ghost, 
                'tol': solver.tol
                }

### PDE
if unsteady:
    if param.pde_type == 'NavierStokes':
        make_target = method.unsteadyNavierStokes
    elif param.pde_type == 'viscousBurgers':
        make_target = method.unsteadyViscousBurgers

else:
    if param.pde_type == 'NavierStokes':
        make_target = method.steadyNavierStokes
    elif param.pde_type == 'viscousBurgers':
        make_target = method.steadyViscousBurgers
    elif param.pde_type == 'Laplace':
        make_target = method.Laplace
